In [1]:

import os
os.chdir("..")

import asyncio
from dotenv import load_dotenv
import shutil
import subprocess
import requests
import time
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
 
load_dotenv()


prompt_template = """Given the following information:

1. User details Json: {user_details_json}


Task:
* If user_details_json is not empty
    * Extract name and relevant past history.
    * Craft a personalized response (max 2 lines) based on the current chat and history.
    * Prioritize using the name and a warm greeting.
* IF JSON DOES NOT EXIST:
    * Craft a general, friendly response (max 2 lines) to continue the conversation.

Example JSON (if exists):
{
  "name": "Alice",
  "past_history": ["Asked about product features", "Expressed interest in discounts"]
}"""


from langchain.prompts import FewShotPromptTemplate
from langchain.prompts import PromptTemplate



prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['user_details_json']
)

text = """{
  "name": "Alice",
  "past_history": ["Asked about product features", "Expressed interest in discounts"]
}"""



llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"))
 



from langchain_community.chat_message_histories import SQLChatMessageHistory, MongoDBChatMessageHistory






In [2]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory


prompt_template = """You are a conversational medical assistant.
Use short, conversational responses as if you're having a live conversation. Restrict
yourself from using unnecessary responses. Keep the responses short and precise.

Given the following information:
User details Json: {user_details_json}


Task:
* Extract name and relevant past history.
* Craft a personalized response (max 2 lines) based on the current chat and history.
* Prioritize using the name and a warm greeting.



"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_template
        ),
        
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm



In [3]:
runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    
    input_messages_key="input",
    history_messages_key="history",
)



In [5]:

runnable_with_history.invoke(
    {"user_details_json":
        """{"name": "Rishi",
        "past_history": ["Asked about product features", "Expressed interest in discounts"]}""",
        
        
     "input": "Hi!"},
    config={"configurable": {"session_id": "2"}},
)


KeyboardInterrupt: 

In [19]:

runnable_with_history.invoke(
    {"user_details_json":
        None,
        
     "input": "My name is rishi and i am having fever!"},
    config={"configurable": {"session_id": "1"}},
)



AIMessage(content="Hello Rishi! I'm here to help. I see you're experiencing a fever, let's discuss your symptoms and how I can assist you today. 😊")

In [20]:
runnable_with_history.invoke(
    {"user_details_json":
        None,
        
     "input": "Can you tell me what questions did i ask"},
    config={"configurable": {"session_id": "2"}},
)



AIMessage(content='I apologize for any confusion, Rishi. Since I don\'t have access to our previous conversations, I can\'t provide you with the specific questions you asked before. However, I\'m here to help you with any questions or concerns you might have now. Is there anything specific you would like to know about our medical platform or any other topic? 😊 \n\n(Note: I\'m assuming you meant "Can you tell me what questions did I ask?" as a new question, so I\'m responding accordingly. If you meant something else, please let me know!) \n\n(Note 2: I\'m repeating my previous response since I\'m not sure if you meant "Can you tell me what questions did I ask?" as a new question or as a confirmation of the previous one. Please let me know if you meant something else!) ')

### MongoDB implementation to store chat history

In [22]:
def get_session_history_mongo(session_id):
    mongo_con_str = os.getenv("MONGODB_URL")
    return MongoDBChatMessageHistory(
        connection_string=mongo_con_str,
        session_id=session_id,
        database_name="messages",
        collection_name="message_history")

In [23]:
runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history_mongo,
    
    input_messages_key="input",
    history_messages_key="history",
)



In [24]:
text = """{
  "name": "Rishi",
  "past_history": ["Asked about llm", "Expressed interest in generative ai"]
}"""
runnable_with_history.invoke({"user_details_json":
        text,
        
     "input": "Hi"},
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content="Hello! I'm here to help. If you have any questions or need information about LLM or generative AI, please let me know. 😊 \n\n(Note: I'm using a default greeting since I don't have your name or past chat history.) \n\nIf you provide me with your user\\_details\\_json, I can personalize the response for you.")

In [10]:
from typing import Dict
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory



class ChatResponse:
    def __init__(self) -> None:
        self.mongo_con_str = os.getenv("MONGODB_URL")

    
    def get_session_history_mongo(self,session_id):
        return MongoDBChatMessageHistory(
            connection_string=self.mongo_con_str,
            session_id=session_id,
            database_name="messages",
            collection_name="message_history")
        
    def get_runnable(self):
        prompt_template = """You are a conversational medical assistant.
                Use short, conversational responses as if you're having a live conversation. Restrict
                yourself from using unnecessary responses. Keep the responses short and precise.

                Given the following information:
                User details Json: {user_details_json}


                Task:
                * Extract name and relevant past history.
                * Craft a personalized response with the user's first name (max 2 lines) based on the current chat and history.
                * The name might be in user_details_json or in message_history.
                * Prioritize using the name and a warm greeting.
                
                 
                """

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    prompt_template
                ),
                
                MessagesPlaceholder(variable_name="history"),
                ("human", "{input}"),
            ]
        )

        runnable = prompt | llm
        
        return runnable
        
    
    def chat(self, 
             user_details_json: Dict,
             input_text: str,
             session_id: str,
             ):
        
        runnable = self.get_runnable()
        runnable_with_history = RunnableWithMessageHistory(
                                                runnable,
                                                self.get_session_history_mongo,
                                                
                                                input_messages_key="input",
                                                history_messages_key="history",
                                                )
        
        
        return runnable_with_history.invoke({"user_details_json":
            user_details_json,
            
        "input": input_text},
        config={"configurable": {"session_id": session_id}},
    )



In [11]:
chat_response = ChatResponse()
chat_response.chat(
    user_details_json={"name": "Rishi",
        "past_history": ["Asked about llm", "Expressed interest in generative ai"]},
    input_text="Hi",
    session_id="1",
    
)

AIMessage(content="Hello! It's nice to see you again. I'm here to provide information and support. If you have any questions about LLM or generative AI, please let me know. 😊")

In [12]:
chat_response.chat(
    user_details_json=None,
    input_text="Hi",
    session_id="1",
    
)

AIMessage(content="Hello! I'm glad you're here. I can help you with any questions or concerns related to LLM or generative AI. What can I assist you with today? 😊 \n\n(Note: I'm using a default greeting since I don't have your name or past chat history.)")

In [6]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

demo_ephemeral_chat_history.add_ai_message("J'adore la programmation.")

demo_ephemeral_chat_history.messages

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

chain_with_message_history.invoke(
    {"input": "What's my name?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content="I'm sorry, I don't have access to that information.")

In [7]:
from langchain_core.runnables import RunnablePassthrough


def trim_messages(chain_input):
    stored_messages = demo_ephemeral_chat_history.messages
    if len(stored_messages) <= 2:
        return False

    demo_ephemeral_chat_history.clear()

    for message in stored_messages[-2:]:
        demo_ephemeral_chat_history.add_message(message)

    return True


chain_with_trimming = (
    RunnablePassthrough.assign(messages_trimmed=trim_messages)
    | chain_with_message_history
)